# Summary of Findings

There is an inverse relationship between per student budget and overall performance.  Schools with a per student budget less than $615 have a 95% overall passing rate while schools with per student budgets of \\$615 to \\$675 have overall passing rates of 73%-77%.

School size also impacts performance of students.  Small and medium schools (2,000 students or less) have an overall passing rate of 95%.  Large schools with more than 2,000 students have a far lower overall passing rate of 75%.

Student success rate has a direct correlation to type of school.  Charter schools boast an overall passing rate of 95% while District schools are pushing 74% passing rate.  The top 5 performing schools are all Charter schools while the bottom 5 performing schools are all District schools.

Trends show that smaller Charter schools have higher success in overall performance of their students than larger District schools.  District schools tend to be larger in size and despite having larger per student budgets, have a much lower overall passing rate than Charter schools.

In [1]:
# dependencies
import pandas as pd

# files to import
schools_file = 'Resources/schools_complete.csv'
students_file = 'Resources/students_complete.csv'

# import data into dataframes
df_schools = pd.read_csv(schools_file, index_col='School ID')
df_students = pd.read_csv(students_file, index_col='Student ID')


### District Summary


Measurement of Districts Key Metrics:

* Total Number of Schools
* Total Number of Students
* Total District Budget
* Average Student Math Score
* Average Student Reading Score
* Percent of Students Passing Math (Score of 70 or Greater)
* Percent of Students Passing Reading (Score of 70 or Greater)
* Overall Passing Rate (Average of Math and Reading Passing Rates)

In [2]:
# District Summary calculations
# Note: this works and looks great, but can't help but think there's a better way to get this into a dataframe
school_count = len(df_schools)
student_count = len(df_students)
total_budget = df_schools['budget'].sum()
avg_math_score = df_students['math_score'].mean()
avg_reading_score = df_students['reading_score'].mean()
avg_overall_score = (avg_math_score + avg_reading_score) / 2

passing_math_cond = df_students['math_score'] >= 70
students_passing_math_count = len(df_students[passing_math_cond])
pct_students_passing_math = (students_passing_math_count / student_count) * 100

passing_reading_cond = df_students['reading_score'] >= 79
students_passing_reading_count = len(df_students[passing_reading_cond])
pct_students_passing_reading = (students_passing_reading_count / student_count) * 100


# Districts Summary DataFrame
df_district_summary = pd.DataFrame({
    'Total Schools' : [school_count],
    'Total Students' : [student_count],
    'Total Budget' : [total_budget],
    'Avg Math Score' : [avg_math_score],
    'Avg Reading Score' : [avg_reading_score],
    '% Passing Math' : [pct_students_passing_math],
    '% Passing Reading' : [pct_students_passing_reading],
    '% Overall Passing Rate' : [avg_overall_score]
})

# Format the summary data
df_district_summary['Total Students'] = df_district_summary['Total Students'].map("{:,}".format)
df_district_summary['Total Budget'] = df_district_summary['Total Budget'].map("${:,.0f}".format)
df_district_summary[['Avg Math Score', 'Avg Reading Score']] = df_district_summary[['Avg Math Score', 'Avg Reading Score']].applymap('{:.2f}'.format)
df_district_summary[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']] =df_district_summary[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']].applymap('{:.2f}%'.format)

# Show summary but hide the index
df_district_summary.style.hide_index()


Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
15,"39,170","$24,649,428",78.99,81.88,74.98%,60.24%,80.43%


### School Summary

Measurement of Key Metrics by School:
  * School Name
  * School Type
  * Total Number of Students
  * Total School Budget
  * Per Student Budget
  * Average Student Math Score
  * Average Student Reading Score
  * Percent of Students Passing Math (Score of 70 or Greater)
  * Percent of Students Passing Reading (sore of 70 or Greater)
  * Overall Passing Rate (Average of Math and Reading Passing Rates)

In [3]:
# function that will return count of passing scores
def count_passed_class(values):
    passed = [
        1
        for x in values
        if x >= 70
    ]
    return len(passed)

# function that will calc percent that passed a class
def percent_passed_class(count_passed_class, total_student_count):
    return (count_passed_class / total_student_count) * 100


In [4]:
# School Summary Calculations

# create a new data frame for school stats
df_school_stats = df_schools.copy()

# add new column for avg budget by student
df_school_stats['per_student_budget'] = (df_school_stats['budget'] / df_school_stats['size'])

# let's give some prettier names to the columns of our data frame
columns ={'school_name' : 'School Name', 
          'type': 'School Type', 
          'size' : 'Total Students', 
          'budget' : 'Total Budget',
          'per_student_budget' : 'Per Student Budget'
         }
df_school_stats = df_school_stats.rename(columns, axis='columns')

# set index to school name - will help with future merge
df_school_stats.set_index(['School Name'], drop=True, inplace=True)


# create new dataframe for student stats by school
df_student_stats=df_students.groupby('school_name').agg({'math_score' : ['mean', count_passed_class], 
                                        'reading_score' : ['mean', count_passed_class], 'student_name': 'count', 
                                       })

# let's flatten our data frame and give columns better names
df_student_stats.columns = df_student_stats.columns.map('_'.join)

# add remaining summary data to dataframe
df_student_stats['pct_passing_math'] = percent_passed_class(df_student_stats['math_score_count_passed_class'], df_student_stats['student_name_count'])
df_student_stats['pct_passing_reading'] = percent_passed_class(df_student_stats['reading_score_count_passed_class'], df_student_stats['student_name_count'])
df_student_stats['pct_overall_passing'] = ((df_student_stats['pct_passing_math'] + df_student_stats['pct_passing_reading']) / 2)

# let's give some prettier names to the columns of our data frame
columns ={'math_score_mean' : 'Avg Math Score', 
          'reading_score_mean' : 'Avg Reading Score', 
          'pct_passing_math': '% Passing Math', 
          'pct_passing_reading': '% Passing Reading', 
          'pct_overall_passing': '% Overall Passing Rate'
         }
df_student_stats = df_student_stats.rename(columns, axis='columns')

# Merge the two dataframes into the final School Summary output only keeping fields we need from student stats
keep_student_stats =['Avg Math Score', 
                     'Avg Reading Score', 
                     '% Passing Math', 
                     '% Passing Reading', 
                     '% Overall Passing Rate']

df_school_summary = pd.merge(df_school_stats, 
                             df_student_stats[keep_student_stats], 
                             how='left', 
                             right_on=['school_name'], 
                             left_index=True)

# let's make sure our index has a nicely formatted name
df_school_summary.index.name = 'School Name'

# Format the summary data into a new dataframe (we'll reuse the unformatted version later)
df_school_summary_formatted = df_school_summary.copy()
df_school_summary_formatted['Total Students'] = df_school_summary_formatted['Total Students'].map("{:,}".format)
df_school_summary_formatted[['Total Budget', 'Per Student Budget']] = df_school_summary_formatted[['Total Budget', 'Per Student Budget']].applymap('${:,.0f}'.format)
df_school_summary_formatted[['Avg Math Score', 'Avg Reading Score']] = df_school_summary_formatted[['Avg Math Score', 'Avg Reading Score']].applymap('{:.2f}'.format)
df_school_summary_formatted[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']] =df_school_summary_formatted[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']].applymap('{:.2f}%'.format)

# Show summary
df_school_summary_formatted.sort_values('School Name', inplace=True)
df_school_summary_formatted

,School Type,Total Students,Total Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,73.64%


### Top Performing Schools (By Passing Rate)

Top 5 performing schools based on Overall Passing Rate and their metrics

In [5]:
# sort the schools summary data by overall passing rate to find the top performing schools
df_school_summary_formatted.sort_values(['% Overall Passing Rate', 'School Name'], ascending=False).head(5)

,School Type,Total Students,Total Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,95.20%


### Bottom Performing Schools (By Passing Rate)

Bottom 5 perform schools based on Overall Passing Rate and their metrics.

In [6]:
# sort the schools summary data by overall passing rate to find the bottom performing schools
df_school_summary_formatted.sort_values(['% Overall Passing Rate', 'School Name'], ascending=True).head(5)


,School Type,Total Students,Total Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,73.80%


### Math Scores by Grade

Average Math Score for students of each grade level (9th, 10th, 11th, 12th) by school.

In [7]:
# let's get the average of all math scores for each school by grade
# with two group by's, let's unstack the data to get it a preferred display format
df_math_group = df_students['math_score'].groupby([df_students['school_name'], df_students['grade']]).mean().unstack()

# rearrange the columns to get them in the right order and get rid of the axis names
df_math_group = df_math_group[['9th', '10th', '11th', '12th']]
df_math_group = df_math_group.rename_axis([None])

# apply some formatting
df_math_group[['9th', '10th', '11th', '12th']] = df_math_group[['9th', '10th', '11th', '12th']].applymap('{:.2f}'.format)

#update column headers to be a bit more description
math_column_names = ['9th Grade Math Avg', '10th Grade Math Avg', '11th Grade Math Avg', '12th Grade Math Avg']
df_math_group.columns = math_column_names

df_math_group

,9th Grade Math Avg,10th Grade Math Avg,11th Grade Math Avg,12th Grade Math Avg
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


### Reading Scores by Grade

Average Reading Score for students of each grade level (9th, 10th, 11th, 12th) by school.

In [8]:
# let's get the average of all math scores for each school by grade
# with two group by's, let's unstack the data to get it a preferred display format
df_reading_group = df_students['reading_score'].groupby([df_students['school_name'], df_students['grade']]).mean().unstack()

# rearrange the columns to get them in the right order and get rid of the axis names
df_reading_group = df_reading_group[['9th', '10th', '11th', '12th']]
df_reading_group = df_reading_group.rename_axis([None])

# apply some formatting
df_reading_group[['9th', '10th', '11th', '12th']] = df_reading_group[['9th', '10th', '11th', '12th']].applymap('{:.2f}'.format)

#update column headers to be a bit more description
reading_column_names = ['9th Grade Reading Avg', '10th Grade Reading Avg', '11th Grade Reading Avg', '12th Grade Reading Avg']
df_reading_group.columns = reading_column_names

df_reading_group

,9th Grade Reading Avg,10th Grade Reading Avg,11th Grade Reading Avg,12th Grade Reading Avg
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

School performance based on Per Student Budgets.

In [9]:
#merge students and schools together
df_combined = pd.merge(df_schools, df_students, how='left', on='school_name')

# add a new field for per student budget
df_combined['per_student_budget'] = df_combined['budget'] / df_combined['size']

# create bins for per student budget ranges
spending_bins = [0, 585, 615, 645, 675]
group_names = ["Under $585", "$585-615", "$615-645", "$645-675"]

# create a cut of the combined data by the per student budget
ds_school_spend_group = pd.cut(df_combined["per_student_budget"], spending_bins, labels=group_names)

# add this series back into combined dataframe
df_combined["per_student_budget_group"] = ds_school_spend_group

# create new dataframe for student stats by per student budget group
df_budget_group_stats=df_combined.groupby('per_student_budget_group').agg({'math_score' : ['mean', count_passed_class], 
                                        'reading_score' : ['mean', count_passed_class], 'student_name': 'count', 
                                       })
# let's flatten our data frame and give columns better names
df_budget_group_stats.columns = df_budget_group_stats.columns.map('_'.join)

#add in the additional data we need
df_budget_group_stats['pct_passing_math'] = percent_passed_class(df_budget_group_stats['math_score_count_passed_class'], df_budget_group_stats['student_name_count'])
df_budget_group_stats['pct_passing_reading'] = percent_passed_class(df_budget_group_stats['reading_score_count_passed_class'], df_budget_group_stats['student_name_count'])
df_budget_group_stats['pct_overall_passing'] = ((df_budget_group_stats['pct_passing_math'] + df_budget_group_stats['pct_passing_reading']) / 2)


#give prettier column names
columns ={'math_score_mean' : 'Avg Math Score', 
          'reading_score_mean' : 'Avg Reading Score', 
          'pct_passing_math': '% Passing Math', 
          'pct_passing_reading': '% Passing Reading', 
          'pct_overall_passing': '% Overall Passing Rate'
         }
df_budget_group_stats = df_budget_group_stats.rename(columns, axis='columns')

# update index header
df_budget_group_stats.index.name='Per Student Budget'

# apply some formatting
df_budget_group_stats[['Avg Math Score', 'Avg Reading Score']] = df_budget_group_stats[['Avg Math Score', 'Avg Reading Score']].applymap('{:.2f}'.format)
df_budget_group_stats[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']] =df_budget_group_stats[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']].applymap('{:.2f}%'.format)

df_budget_group_stats[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
# df_budget_group_stats[['math_score_mean', 'reading_score_mean']]

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
Under $585,83.36,83.96,93.70%,96.69%,95.19%
$585-615,83.53,83.84,94.12%,95.89%,95.01%
$615-645,78.06,81.43,71.40%,83.61%,77.51%
$645-675,77.05,81.01,66.23%,81.11%,73.67%


## Scores by School Size

School performance based on school size.

In [10]:
# create bins for school size ranges
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (Under 1,000)", "Medium (1,000-2,000)", "Large (2,000-5,000)"]

df_combined

# create a cut of the combined data per school size
ds_school_size_group = pd.cut(df_combined["size"], size_bins, labels=group_names)

# add this series back into combined dataframe
df_combined["school_size_group"] = ds_school_size_group


# create new dataframe for student stats by per student budget group
df_size_group_stats=df_combined.groupby('school_size_group').agg({'math_score' : ['mean', count_passed_class], 
                                        'reading_score' : ['mean', count_passed_class], 'student_name': 'count', 
                                       })
# let's flatten our data frame and give columns better names
df_size_group_stats.columns = df_size_group_stats.columns.map('_'.join)

#add in the additional data we need
df_size_group_stats['pct_passing_math'] = percent_passed_class(df_size_group_stats['math_score_count_passed_class'],  df_size_group_stats['student_name_count'])
df_size_group_stats['pct_passing_reading'] = percent_passed_class(df_size_group_stats['reading_score_count_passed_class'],  df_size_group_stats['student_name_count'])
df_size_group_stats['pct_overall_passing'] = ((df_size_group_stats['pct_passing_math'] + df_size_group_stats['pct_passing_reading']) / 2)


#give prettier column names
columns ={'math_score_mean' : 'Avg Math Score', 
          'reading_score_mean' : 'Avg Reading Score', 
          'pct_passing_math': '% Passing Math', 
          'pct_passing_reading': '% Passing Reading', 
          'pct_overall_passing': '% Overall Passing Rate'
         }
df_size_group_stats = df_size_group_stats.rename(columns, axis='columns')

df_size_group_stats.index.name='School Size (# of Students)'

# apply some formatting
df_size_group_stats[['Avg Math Score', 'Avg Reading Score']] = df_size_group_stats[['Avg Math Score', 'Avg Reading Score']].applymap('{:.2f}'.format)
df_size_group_stats[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']] =df_size_group_stats[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']].applymap('{:.2f}%'.format)

df_size_group_stats[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
# df_budget_group_stats[['math_score_mean', 'reading_score_mean']]


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size (# of Students),,,,,
"Small (Under 1,000)",83.83,83.97,93.95%,96.04%,95.00%
"Medium (1,000-2,000)",83.37,83.87,93.62%,96.77%,95.19%
"Large (2,000-5,000)",77.48,81.20,68.65%,82.13%,75.39%


## Scores by School Type

School performance based on School Type.

In [11]:
df_combined

# create new dataframe for student stats by per student budget group
df_type_group_stats=df_combined.groupby('type').agg({'math_score' : ['mean', count_passed_class], 
                                        'reading_score' : ['mean', count_passed_class], 'student_name': 'count', 
                                       })
# let's flatten our data frame and give columns better names
df_type_group_stats.columns = df_type_group_stats.columns.map('_'.join)

#add in the additional data we need
df_type_group_stats['pct_passing_math'] = percent_passed_class(df_type_group_stats['math_score_count_passed_class'],  df_type_group_stats['student_name_count'])
df_type_group_stats['pct_passing_reading'] = percent_passed_class(df_type_group_stats['reading_score_count_passed_class'],  df_type_group_stats['student_name_count'])
df_type_group_stats['pct_overall_passing'] = ((df_type_group_stats['pct_passing_math'] + df_type_group_stats['pct_passing_reading']) / 2)


#give prettier column names
columns ={'math_score_mean' : 'Avg Math Score', 
          'reading_score_mean' : 'Avg Reading Score', 
          'pct_passing_math': '% Passing Math', 
          'pct_passing_reading': '% Passing Reading', 
          'pct_overall_passing': '% Overall Passing Rate'
         }
df_type_group_stats = df_type_group_stats.rename(columns, axis='columns')

df_type_group_stats
df_type_group_stats.index.name='School Type'


df_type_group_stats[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# apply some formatting
df_type_group_stats[['Avg Math Score', 'Avg Reading Score']] = df_type_group_stats[['Avg Math Score', 'Avg Reading Score']].applymap('{:.2f}'.format)
df_type_group_stats[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']] =df_type_group_stats[['% Passing Math', '% Passing Reading', '% Overall Passing Rate']].applymap('{:.2f}%'.format)

df_type_group_stats[['Avg Math Score', 'Avg Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
# df_budget_group_stats[['math_score_mean', 'reading_score_mean']]

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%
